# Background on image moments in scikit-image

The moments of a shape are particular weighted averages (in statistics, "moments")
of the pixel intensities. In physics they're the distribution of matter about
a point or an axis etc.

They are usually described as a function $f(x,y)$ which can just be read as
_the value (or intensity) of the pixel in an image at position $(x,y)$_, so f is the 'image function'.

If the image is binary (e.g. a binary blob), then we'd say the image function f takes
values in [0,1].

A moment of order $p + q$ is defined for a 2D continuous function on a region as:

$$M_{pq} = \int x^p y^q f(x,y) dx dy$$

So if we talk about _"first order moments"_ then we mean $M_{01}$, $M_{10}$, 

$$M_{01} = \int y f(x,y) dx dy$$
$$M_{10} = \int x f(x,y) dx dy$$

and by _"second order moments"_ we mean $M_{11}$, $M_{20}$, $M_{02}$

Since we have discrete pixels not a continuous function, in fact the integrals
just become summations over x and y. So in other words: moments $M_{pq}$ counts
the pixels over an image $f(x,y)$ and the $0^{th}$ moment is just the total count
of pixels in an image, i.e. its area (it's usually called the area moment for binary
images or otherwise the 'mass' of the image for grayscale etc.).

Image moments are sensitive to position, but by subtracting the average x
and y position in the above equation we can obtain a translation-invariant
"central moment".

$$μ_pq = \int {(x-\bar{x})}^p {(y-\bar{y})}^q f(x,y) dx dy$$

where

$$\bar{x} = \frac{M_{10}}{M_{00}} \hspace{10pt}and\hspace{10pt} \bar{y} = \frac{M_{01}}{M_{00}}$$

Meaning first order x or y moment divided by the area moment

- $(\bar{x},\bar{y})$ is the centroid (a.k.a. centre of gravity)

So far so simple.

Second order moments $M_{20}$ and $M_{02}$ describe the "distribution of mass"
of the image with respect to the coordinate axes. In mechanics they're the
_moments of inertia_.

Another mechanical quality is the _radius of gyration_
with respect to an axis, expressed as 

$$\sqrt{\frac{M_{20}}{M_{00}}} \hspace{10pt}and\hspace{10pt} \sqrt{\frac{M_{02}}{M_{00}}}$$

There are others: scale invariant moments and rotation invariant moments
(the latter are a.k.a. Hu moments)

The one of interest for the code in this repo is a little more complicated.

## Orientation information from image moments

The covariance matrix of an image can be obtained from second order central moments as below
(credit: [Wikipedia](https://en.wikipedia.org/wiki/Image_moment#Examples_2)

$$\mu^\prime_{20} = \frac{\mu_{20}}{\mu_{00}} = \frac{M_{20}}{M_{00}} - \bar{x}^2 $$

$$\mu^\prime_{02} = \frac{\mu_{02}}{\mu_{00}} = \frac{M_{02}}{M_{00}} - \bar{y}^2 $$

$$\mu^\prime_{11} = \frac{\mu_{11}}{\mu_{00}} = \frac{M_{11}}{M_{00}} - \bar{x}\bar{y} $$

The covariance matrix of the image $I(x,y)$ is then:

$$cov[I(x,y)] = \begin{bmatrix}
\mu^\prime_{20} &\mu^\prime_{11}\\
\mu^\prime_{02} &\mu^\prime_{11}
\end{bmatrix}$$

[![](https://raw.githubusercontent.com/lmmx/shots/master/2021/Apr/image-covariance-matrix.png)](https://en.wikipedia.org/wiki/Image_moment#Examples_2)

> The eigenvectors of this matrix correspond to the major and minor axes of the
> image intensity, so the **orientation** can thus be extracted from the angle
> of the eigenvector associated with the largest eigenvalue towards the axis
> closest to this eigenvector...

- See [this article](https://www.visiondummy.com/2014/04/geometric-interpretation-covariance-matrix/)
  for a nice geometric interpretation of covariance, which makes this usage intuitive

## Further info

In fact, the moments above are all _geometric moments_ since the polynomial basis
is a standard power basis $x^p$ multiplied by $y^q$.

If you want to read more, check out
Flusser et al's book _Moments and Moment Invariants in Pattern Recognition_